In [ ]:
#################Calculate precipitation and GDD based on shapefiles

#enter start and end dates
start='2021-01-01'
end='2021-10-24'
#enter the base temperature to use (this is in F)
t_base=32

#read in the shapefile; can replace 
os.chdir('G:\\C\\GEE')
sf = shapefile.Reader("Brookings_LL.shp")

#since its 4km pixel just use coordinates for first polygon in shapefile
s=sf.shape(1)
f=len(s.points)
feature=s.points[0:f]


#function to simplify dictionary
def simplify(fc):

    def feature2dict(f):
        id = f['id']
        out = f['properties']
        out.update(id=id)
        return out
    out = [feature2dict(x) for x in fc['features']]
    return out

#use gridmet data
dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date(start,end))
#get tmax and tmin
temp_max = dataset.select('tmmx')
geometry = ee.Geometry.Polygon(feature)
def _reduce_region(image):     
    stat_dict = image.reduceRegion(ee.Reducer.mean(), geometry);
    return ee.Feature(None, stat_dict)
fc=temp_max.filterBounds(geometry).map(_reduce_region).getInfo()
out = simplify(fc)
GDD_df=pd.DataFrame(out)
GDD_df.columns=['Tmax',"Date"]
GDD_df['Date'] = pd.to_datetime(GDD_df['Date'])
GDD_df['Tmax']=(GDD_df['Tmax']-273.15)*(9/5)+32

dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date(start,end))
t_min = dataset.select('tmmn')
geometry = ee.Geometry.Polygon(feature)
def _reduce_region(image):     
    stat_dict = image.reduceRegion(ee.Reducer.mean(), geometry);
    return ee.Feature(None, stat_dict)
t_min=t_min.filterBounds(geometry).map(_reduce_region).getInfo()
t_min = simplify(t_min)
t_min=pd.DataFrame(t_min)

#
GDD_df['Tmin']=t_min['tmmn']
GDD_df['Tmin']=(GDD_df['Tmin']-273.15)*(9/5)+32
GDD_df['GDU']=((GDD_df['Tmin']+GDD_df['Tmax'])/2) -t_base
GDD_df['GDU'] = np.where(GDD_df['GDU'] < 0, 0, GDD_df['GDU'])
GDD_df['GDU'] = np.where(GDD_df['Date'] < '2021-03-15', 0, GDD_df['GDU'])
GDD_df["GDU_sum"]=GDD_df.GDU.cumsum()
GDD_df['Month'] = pd.DatetimeIndex(GDD_df['Date']).month
GDD_df['Week'] = pd.DatetimeIndex(GDD_df['Date']).week
GDD_df['year'] = pd.DatetimeIndex(GDD_df['Date']).year


#get precipitation
dataset = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filter(ee.Filter.date(start,end))
ppt = dataset.select('pr')
geometry = ee.Geometry.Polygon(feature)
def _reduce_region(image):     
    stat_dict = image.reduceRegion(ee.Reducer.mean(), geometry);
    return ee.Feature(None, stat_dict)
ppt=ppt.filterBounds(geometry).map(_reduce_region).getInfo()
ppt = simplify(ppt)
ppt=pd.DataFrame(ppt)
ppt.columns=['Ppt_mm',"Date"]

GDD_df['Ppt_mm']=ppt['Ppt_mm']
#GDD_df['Ppt_in']=ppt['Ppt_mm']*0.0393701
GDD_df['PPT_sum']=GDD_df.Ppt_mm.cumsum()

GDD_df
#GDD_df.to_csv('Brookings_Gridmet.csv')